**REMARKS**

In this section, we will learn how to summarize our text with python. Let's learn.

In [ ]:
# Install useful library
!pip install spacy

In [58]:
# Prepare library
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords = set(stopwords.words('indonesian'))
from string import punctuation
import spacy

# importing libraries
import re 
import string
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.probability import FreqDist
import unicodedata

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [75]:
import pandas as pd
nlp = spacy.blank('id')
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [10]:
# Prepare dataset
data = pd.read_csv('classification.csv',sep=';', encoding='ISO-8859-1')
data = data.drop(['articlename'], axis=1)
data.head()

,textdata,Amar
0,putusan nomor put000082162018ppmxva tahun 2...,Kabul
1,putusan nomor put000083162018ppmxva tahun 20...,Kabul
2,putusan nomor put 000215162018ppmiiia tahun ...,Kabul
3,putusan nomor put000216162018ppmiiia tahun ...,Kabul
4,putusan nomor put 000838162018ppmxb tahun 20...,Kabul


In [43]:
# Process Cleaning Function
def remove_text_special(text):
  # remove tab, new line, ans back slice
  text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
  # remove non ASCII (emoticon, chinese word, .etc)
  text = text.encode('ascii', 'replace').decode('ascii')
  # remove mention, link, hashtag
  text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
  text = re.sub("(.)\\1{2,}", "\\1", text)
  # remove incomplete URL
  return text.replace("http://", " ").replace("https://", " ")

def remove_number(text):
  return  re.sub(r"\d+", "", text)

def remove_whitespace_LT(text):
  return text.strip()

def remove_whitespace_multiple(text):
  return re.sub('\s+',' ',text)

def remove_single_char(text):
  return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_single_first(text):
  return re.sub("\^[a-zA-Z]\s+", " ", text)

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def remove_unuseful_text(text):
  text = text.replace("pajak", " ")
  text = text.replace("yang", " ")
  text = text.replace("dan", " ")
  text = text.replace("000", " ")
  text = text.replace("dengan", " ")
  text = text.replace("the", " ")
  text = text.replace("tanggal", " ")
  text = text.replace("of", " ")
  text = text.replace("pt", " ")
  text = text.replace("1", " ")
  text = text.replace("2", " ")
  text = text.replace("tahun", " ")
  text = text.replace("nomor", " ")
  text = text.replace("0", " ")
  
  return text

def alternative_review_messages(msg):
  stopwords = set(stopwords.words('indonesian'))
  msg = msg.lower()
  nltk_pos = [tag[1] for tag in pos_tag(word_tokenize(msg))]
  msg = [tag[0] for tag in pos_tag(word_tokenize(msg))]
  wnpos = ['a' if tag[0] == 'J' else tag[0].lower() if tag[0] in ['N', 'R', 'V'] else 'n' for tag in nltk_pos]
  msg = " ".join([lemmatizer.lemmatize(word, wnpos[i]) for i, word in enumerate(msg)])
  # removing stopwords 
  msg = [word for word in msg.split() if word not in stopwords]
  
  return msg

In [51]:
def get_clean(x):
  x = str(x).lower().replace('\\', '').replace('_', ' ')
  x = remove_text_special(x)
  x = remove_number(x)
  x = remove_whitespace_LT(x)
  x = remove_whitespace_multiple(x)
  x = remove_single_char(x)
  x = remove_single_first(x)
  x = remove_accented_chars(x)
  x = remove_unuseful_text(x)
  # x = TextBlob(x).correct()
  return x

In [45]:
text = data.iloc[1,:][0]
text

'putusan  nomor  put000083162018ppmxva tahun 2019   demi keadilan berdasarkan ketuhanan yang maha esa   pengadilan pajak   memeriksa dan mengadili sengketa pajak dengan acara biasa pada tingkat pertama  dan terakhir terhadap keputusan direktur jenderal pajak nomor kep00085kebwpj012017  tanggal 11 oktober 2017 tentang keberatan wajib pajak atas surat ketetapan pajak lebih  bayar pajak pertambahan nilai barang dan jasa masa pajak juli 2015 nomor  000204071512317 tanggal 25 januari 2017 yang terdaftar dalam berkas sengketa nomor  000083162018pp telah mengambil putusan sebagai berikut dalam sengketa antara   izin kuasa hukum \t kep151ppikh2016 tanggal 27 juni 2016 yang diperpanjang  dengan kep193ppikh2018 tanggal 29 juni 2018   surat kuasa khusus nomor  dsippox022205 18 tanggal 02 mei 2018   izin kuasa hukum \t kep122ppikh2016 tanggal 15 juni 2016 yang diperpanjang  dengan kep167ppikh2018 tanggal 31 mei 2018   surat kuasa khusus nomor  dsippox021505 18 tanggal 02 mei 2018   izin kuasa huku

In [46]:
clean = get_clean(text) #processing needs 8m 25s with TEXT Blob

In [52]:
clean

TextBlob("putusan   putppmxva   semi keadilan berdasarkan ketuhanan   masha sea pengadilan   memeriksa   mengadili sengketa     care bias papa tingkat pertama   terakhir terhadap keputusan director general     kepkebwpj   october testing keberatan wait   matas murat ketetapan   leash bear   pertambahan fili baring   vasa mass   julie     january   terdaftar calm berks sengketa   pp tell mengambil putusan sebagai berikut calm sengketa intra in bursa hum kepppikh   june   diperpanjang   kepppikh   june murat bursa husks   dsippox   me in bursa hum kepppikh   june   diperpanjang   kepppikh   me murat bursa husks   dsippox   me in bursa hum kepppikh   augustus   diperpan sang   kepppikh   augustus murat bursa husks   dsippox   me berdasarkan murat bursa tersebut siberia bursa ole berdasarkan ali at isaiah rapt until selanjutnya debut sebagai pemohon bending  lawn berdasarkan suratsurat texas terakhir   steps   augustus   until selanjutnya debut sebagai terbanding pengadilan   tersebut tell

In [54]:
s = get_clean(text)
s

'putusan   putppmxva   demi keadilan berdasarkan ketuhanan   maha esa pengadilan   memeriksa   mengadili sengketa     acara biasa pada tingkat pertama   terakhir terhadap keputusan direktur jenderal     kepkebwpj   oktober tentang keberatan wajib   atas surat ketetapan   lebih bayar   pertambahan nilai barang   jasa masa   juli     januari   terdaftar dalam berkas sengketa   pp telah mengambil putusan sebagai berikut dalam sengketa antara izin kuasa hukum kepppikh   juni   diperpanjang   kepppikh   juni surat kuasa khusus   dsippox   mei izin kuasa hukum kepppikh   juni   diperpanjang   kepppikh   mei surat kuasa khusus   dsippox   mei izin kuasa hukum kepppikh   agustus   diperpan jang   kepppikh   agustus surat kuasa khusus   dsippox   mei berdasarkan surat kuasa tersebut diberi kuasa oleh berdasarkan asli akta risalah rapat untuk selanjutnya disebut sebagai pemohon banding  elawan berdasarkan suratsurat tugas terakhir   stpj   agustus   untuk selanjutnya disebut sebagai terbanding p

In [82]:
# Build an NLP Object
docx = nlp(s)
# Tokenization of Text
mytokens = [token.text for token in docx]

In [60]:
stopwords = set(stopwords.words('indonesian'))

In [83]:
word_frequencies = {}
for word in docx:
    if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

In [84]:
word_frequencies

{' ': 111,
 '  ': 495,
 '   ': 38,
 '    ': 65,
 '     ': 1,
 '      ': 3,
 '?': 5,
 'aaaaa': 1,
 'ab': 2,
 'acara': 1,
 'acf': 1,
 'ad': 1,
 'administrasi': 4,
 'agaimana': 1,
 'agustus': 5,
 'ahuah': 2,
 'ai': 1,
 'akibat': 2,
 'akta': 5,
 'akte': 1,
 'aktiva': 2,
 'alamat': 3,
 'alasan': 3,
 'alasanalasan': 2,
 'an': 3,
 'analisa': 2,
 'anggaran': 1,
 'apr': 1,
 'april': 10,
 'arn': 1,
 'arus': 2,
 'asasi': 2,
 'asli': 1,
 'awar': 1,
 'ayat': 52,
 'ba': 1,
 'bahan': 2,
 'bahasa': 2,
 'baku': 2,
 'balk': 1,
 'banding': 66,
 'bandingnya': 1,
 'bank': 2,
 'bantahan': 1,
 'barang': 48,
 'barangnya': 4,
 'batas': 1,
 'bayar': 19,
 'bayarseharusnya': 1,
 'bbbbb': 1,
 'bc': 2,
 'be': 2,
 'bea': 3,
 'beaman': 1,
 'bearran': 1,
 'beas': 1,
 'belawan': 4,
 'beli': 2,
 'ber': 1,
 'berasal': 4,
 'berdasarkan': 12,
 'bererar': 1,
 'berhak': 1,
 'berikat': 19,
 'berisi': 1,
 'berkas': 2,
 'berkewajiban': 1,
 'berlaku': 3,
 'bermotor': 1,
 'berproduksi': 1,
 'bersangkutan': 1,
 'bersengketa': 1,
 

In [85]:
# Maximum Word Frequency
maximum_frequency = max(word_frequencies.values())
maximum_frequency

495

In [86]:
for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

In [87]:
# Frequency Table
word_frequencies

{' ': 0.22424242424242424,
 '  ': 1.0,
 '   ': 0.07676767676767676,
 '    ': 0.13131313131313133,
 '     ': 0.00202020202020202,
 '      ': 0.006060606060606061,
 '?': 0.010101010101010102,
 'aaaaa': 0.00202020202020202,
 'ab': 0.00404040404040404,
 'acara': 0.00202020202020202,
 'acf': 0.00202020202020202,
 'ad': 0.00202020202020202,
 'administrasi': 0.00808080808080808,
 'agaimana': 0.00202020202020202,
 'agustus': 0.010101010101010102,
 'ahuah': 0.00404040404040404,
 'ai': 0.00202020202020202,
 'akibat': 0.00404040404040404,
 'akta': 0.010101010101010102,
 'akte': 0.00202020202020202,
 'aktiva': 0.00404040404040404,
 'alamat': 0.006060606060606061,
 'alasan': 0.006060606060606061,
 'alasanalasan': 0.00404040404040404,
 'an': 0.006060606060606061,
 'analisa': 0.00404040404040404,
 'anggaran': 0.00202020202020202,
 'apr': 0.00202020202020202,
 'april': 0.020202020202020204,
 'arn': 0.00202020202020202,
 'arus': 0.00404040404040404,
 'asasi': 0.00404040404040404,
 'asli': 0.00202020202

In [89]:
# Sentence Tokens
sentence_list = [ sentence for sentence in docx.sents ]
sentence_list

[putusan   putppmxva   demi keadilan berdasarkan ketuhanan   maha esa pengadilan   memeriksa   mengadili sengketa     acara biasa pada tingkat pertama   terakhir terhadap keputusan direktur jenderal     kepkebwpj   oktober tentang keberatan wajib   atas surat ketetapan   lebih bayar   pertambahan nilai barang   jasa masa   juli     januari   terdaftar dalam berkas sengketa   pp telah mengambil putusan sebagai berikut dalam sengketa antara izin kuasa hukum kepppikh   juni   diperpanjang   kepppikh   juni surat kuasa khusus   dsippox   mei izin kuasa hukum kepppikh   juni   diperpanjang   kepppikh   mei surat kuasa khusus   dsippox   mei izin kuasa hukum kepppikh   agustus   diperpan jang   kepppikh   agustus surat kuasa khusus   dsippox   mei berdasarkan surat kuasa tersebut diberi kuasa oleh berdasarkan asli akta risalah rapat untuk selanjutnya disebut sebagai pemohon banding  elawan berdasarkan suratsurat tugas terakhir   stpj   agustus   untuk selanjutnya disebut sebagai terbanding p

In [90]:
# Sentence Score via comparrng each word with sentence
sentence_scores = {}  
for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 20:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

In [91]:
# Sentence Score Table
sentence_scores

{  war ai arn omx xn caao  sivi siotf rw sewn  oslif mil roi bea ?: 1.490909090909091,
    acf ?: 0.08888888888888888}

In [26]:
# Build an function
from heapq import nlargest
def text_summerization(text,select_lenth):
  # Build an NLP Object
  docx = nlp(text)
  # Tokenization of Text
  mytokens = [token.text for token in docx]

  # Build Word Frequency
  # word.text is tokenization in spacy
  word_frequencies = {}
  for word in docx:
    if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
  # Maximum Word Frequency
  maximum_frequency = max(word_frequencies.values())
  for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
  # Sentence Tokens
  sentence_list = [ sentence for sentence in docx.sents ]

  # Sentence Score via comparrng each word with sentence
  sentence_scores = {}  
  for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]
  summarized_sentences = nlargest(select_lenth, sentence_scores, key=sentence_scores.get)
  print("Summarized sentences:")
  print(summarized_sentences)

In [27]:
text=text
select_lenth=int(input("enter number sentences you want : "))
text_summerization(text,select_lenth)

enter number sentences you want : 100
Summarized sentences:
[]
